In [55]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("regresion_distribuida") \
       .getOrCreate()

spark


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,107 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [61.2 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,398 kB in 8s (301 kB/s)
Reading package lists... Done
Bu

In [56]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler

In [57]:
spark = SparkSession.builder.appName('regresion_distribuida').getOrCreate()

In [73]:
# IMPORTACIONES NECESARIAS
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline

In [60]:
data = spark.read.csv("Student_Performance.csv", header=True, inferSchema=True)
data.show(5)

+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+
|Hours Studied|Previous Scores|Extracurricular Activities|Sleep Hours|Sample Question Papers Practiced|Performance Index|
+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+
|            7|             99|                       Yes|          9|                               1|             91.0|
|            4|             82|                        No|          4|                               2|             65.0|
|            8|             51|                       Yes|          7|                               2|             45.0|
|            5|             52|                       Yes|          5|                               2|             36.0|
|            7|             75|                        No|          8|                               5|             66.0|
+-------------+---------

In [61]:
data.dtypes

[('Hours Studied', 'int'),
 ('Previous Scores', 'int'),
 ('Extracurricular Activities', 'string'),
 ('Sleep Hours', 'int'),
 ('Sample Question Papers Practiced', 'int'),
 ('Performance Index', 'double')]

In [64]:


def convertir_a_binario(valor):
    return when(valor == 'Yes', 1).otherwise(0)

data = data.withColumn("Extracurricular Activities", convertir_a_binario(data["Extracurricular Activities"]))

data.write.csv("Student_Performance.csv_updated.csv", header=True, mode="overwrite")

data.show(5)

+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+
|Hours Studied|Previous Scores|Extracurricular Activities|Sleep Hours|Sample Question Papers Practiced|Performance Index|
+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+
|            7|             99|                         0|          9|                               1|             91.0|
|            4|             82|                         0|          4|                               2|             65.0|
|            8|             51|                         0|          7|                               2|             45.0|
|            5|             52|                         0|          5|                               2|             36.0|
|            7|             75|                         0|          8|                               5|             66.0|
+-------------+---------

In [65]:
df_original = data.select(*data.columns)

In [71]:
predictor_cols = data.select(col("Hours Studied"),
col("Previous Scores"),
col("Extracurricular Activities"),
col("Sleep Hours"),
col("Sample Question Papers Practiced"),
col("Performance Index"))

In [76]:

assembler = VectorAssembler(
    inputCols=["Previous Scores", "Hours Studied", "Sleep Hours", "Sample Question Papers Practiced", "Extracurricular Activities"],
    outputCol="features"
)

data = assembler.transform(predictor_cols)


In [77]:
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
pipeline = Pipeline(stages=[scaler])
scalerModel = pipeline.fit(data)
scaledData = scalerModel.transform(data)


In [79]:
train_data, test_data = scaledData.randomSplit([0.8, 0.2], seed=53)
scaledData.show(5)

+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+--------------------+--------------------+
|Hours Studied|Previous Scores|Extracurricular Activities|Sleep Hours|Sample Question Papers Practiced|Performance Index|            features|     scaled_features|
+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+--------------------+--------------------+
|            7|             99|                         0|          9|                               1|             91.0|[99.0,7.0,9.0,1.0...|[1.0,0.75,1.0,0.1...|
|            4|             82|                         0|          4|                               2|             65.0|[82.0,4.0,4.0,2.0...|[0.71186440677966...|
|            8|             51|                         0|          7|                               2|             45.0|[51.0,8.0,7.0,2.0...|[0.18644067796610...|
|            5| 

In [80]:
#Regresion

regresion = LinearRegression(featuresCol="scaled_features", labelCol="Performance Index", predictionCol="prediction")
regresion_model = regresion.fit(train_data)
predictions = regresion_model.transform(test_data)


predictions.select("Performance Index", "Prediction").show(5)


+-----------------+------------------+
|Performance Index|        Prediction|
+-----------------+------------------+
|             13.0|12.124187190586605|
|             12.0|13.284021405411275|
|             11.0|13.939454085374482|
|             13.0| 15.27850248354692|
|             14.0|13.232518664831886|
+-----------------+------------------+
only showing top 5 rows



In [81]:
# ROOT MEAN SQUARED ERROR

evaluator = RegressionEvaluator(labelCol="Performance Index", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)

print("Root Mean Squared Error (RMSE):", rmse)


Root Mean Squared Error (RMSE): 2.047937763853028
